In [1]:
# # Windows file locations

# Windows_scrapped_data = 'C:\Users\oneti\OneDrive\Documents\GitHub\IMDb_movie_review_sentiment_analysis\20_movies_datasets_with_Scrapped_data'
# Windows_URL_location = 'C:\Users\oneti\OneDrive\Documents\GitHub\IMDb_movie_review_sentiment_analysis\20_movies_datasets_with_URL'
# Windows_movies_location = 'C:\Users\oneti\OneDrive\Documents\GitHub\IMDb_movie_review_sentiment_analysis\20_movies_datasets'

# Mac file locations


SyntaxError: invalid syntax (212377264.py, line 3)

# Getting a list of movies 

In [1]:
import pandas as pd

In [2]:
# Adjust these file paths according to where you've saved the downloaded datasets
basics_path = '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/Data/IMDb title basics.tsv'
ratings_path = '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/Data/IMDb Title Ratings.tsv'

In [3]:
# Reading the basics file with the correct column names
basics_df = pd.read_csv(
    basics_path, 
    sep='\t', 
    low_memory=False, 
    usecols=['tconst', 'titleType', 'primaryTitle', 'genres', 'isAdult', 'startYear', 'runtimeMinutes']
)

In [4]:
# Filtering for movies
movies_df = basics_df[basics_df['titleType'] == 'movie']

In [5]:
# Reading the ratings file
ratings_df = pd.read_csv(ratings_path, sep='\t', usecols=['tconst', 'averageRating', 'numVotes'])

# Merging the datasets on 'tconst' to combine movie details with ratings
merged_df = pd.merge(movies_df, ratings_df, on='tconst')

# Selecting and renaming the columns
IMDb_movies_df = merged_df[['primaryTitle', 'genres', 'averageRating', 'startYear', 'isAdult', 'runtimeMinutes', 'numVotes']]
IMDb_movies_df.columns = ['Movie Name', 'Genre', 'Rating', 'Release Date', 'isAdult', 'Runtime Minutes', 'numVotes']

In [6]:
# Remove all values from Release date that are not numeric
IMDb_movies_df = IMDb_movies_df[IMDb_movies_df['Release Date'].str.isnumeric()]

In [7]:
# set datatypes for IMDb_movies_df
IMDb_movies_df['Release Date'] = IMDb_movies_df['Release Date'].astype('int')
# IMDb_movies_df['isAdult'] = IMDb_movies_df['isAdult'].astype('bool')
IMDb_movies_df['Runtime Minutes'] = IMDb_movies_df['Runtime Minutes'].apply(pd.to_numeric, errors='coerce')
IMDb_movies_df['numVotes'] = IMDb_movies_df['numVotes'].apply(pd.to_numeric, errors='coerce')

In [8]:
# Filter IMDb_movies_df for Release date > 2015
IMDb_movies_df = IMDb_movies_df[IMDb_movies_df['Release Date'] > 2015]

# Remove all num values from Genre column
IMDb_movies_df['Genre'] = IMDb_movies_df['Genre'].str.replace(r'\d+', '')

In [9]:
# Remove all values from IMDb_movies_df columns that have a value /N
IMDb_movies_df = IMDb_movies_df[IMDb_movies_df['Movie Name'] != '\\N']
IMDb_movies_df = IMDb_movies_df[IMDb_movies_df['Rating'] != '\\N']
IMDb_movies_df = IMDb_movies_df[IMDb_movies_df['isAdult'] != '\\N']
IMDb_movies_df = IMDb_movies_df[IMDb_movies_df['Runtime Minutes'] != '\\N']
IMDb_movies_df = IMDb_movies_df[IMDb_movies_df['numVotes'] != '\\N']
IMDb_movies_df = IMDb_movies_df[IMDb_movies_df['Rating'] != '\\N']
IMDb_movies_df = IMDb_movies_df[IMDb_movies_df['Genre'] != '\\N']

In [10]:
# Filter any movies with less than 10000 votes
IMDb_movies_df = IMDb_movies_df[IMDb_movies_df['numVotes'] > 10000]

In [11]:
# Remove all rows where the Movie Name, Release Date and Runtime Minutes are duplicated
IMDb_movies_df = IMDb_movies_df.drop_duplicates(subset=['Movie Name', 'Release Date', 'Runtime Minutes'])

In [12]:
# Reset index
IMDb_movies_df.reset_index(drop=True, inplace=True)

In [13]:
IMDb_movies_df.shape

(2667, 7)

### Dividing the Movies dataframe into 40 csv's

In [15]:
import os
import math

# Create the directory
directory = '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/IMDb_movie_review_sentiment_analysis/20_movies_datasets'
if not os.path.exists(directory):
    os.makedirs(directory)

# Calculate the number of rows per chunk
chunk_size = 2667 // 40
remaining_rows = 2667 % 40

# Split the dataframe into chunks and save as separate CSV files
for i in range(40):
    start = i * chunk_size
    end = start + chunk_size
    if i == 39:  # Last chunk
        end += remaining_rows
    df = IMDb_movies_df.iloc[start:end]
    df.to_csv(f'{directory}/IMDb_movies_df_{i + 1}.csv', index=False)

## Importing all Movies CSV
-----

In [16]:
import pandas as pd
import os

# Define the directory where the CSV files are located
directory = '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/IMDb_movie_review_sentiment_analysis/20_movies_datasets'

# Initialize an empty dictionary to store the dataframes
dataframes = {}

# Loop through all the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Remove the '.csv' extension and use the filename as the dictionary key
        dataframe_name = filename.replace('.csv', '')
        # Store the DataFrame in the dictionary
        dataframes[dataframe_name] = df

In [17]:
# Defining all dataframes as variables
for i in range(1, 41):
    globals()[f"imdb_movies_df_{i}"] = dataframes[f"IMDb_movies_df_{i}"]

---

# URL Extracting script for all dataframes

In [376]:
# Working URL extracting sctipt for all 20 dataframes

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Setting up Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")  # Headless mode to run without opening browser window
service = Service('path_to_chromedriver')  # Replace 'path_to_chromedriver' with the actual path
# Setup WebDriver (example with Chrome)
driver = webdriver.Chrome()

# Function to get the URL of User Reviews for a movie
def get_user_reviews_url(movie_name, release_date):
    try:
        # Open IMDb website
        driver.get("https://www.imdb.com/")
        time.sleep(4)  # Wait for page to load

        # Find search box and input movie name
        search_box = driver.find_element(By.ID, "suggestion-search")
        search_box.clear()
        search_box.send_keys(movie_name)
        search_box.send_keys(Keys.RETURN)
        time.sleep(4)  # Wait for search results to load

        # Find and click on the release year
        release_year_element = driver.find_element(By.XPATH, f"//*[contains(text(), '{release_date}')]")
        release_year_element.click()
        time.sleep(4)  # Wait for page to load

        # Find and click on User Reviews button
        user_reviews_button = driver.find_element(By.XPATH, "//a[contains(text(), 'User reviews')]")
        user_reviews_url = user_reviews_button.get_attribute('href')
        return user_reviews_url

    except Exception as e:
        print(f"Error occurred for {movie_name}: {str(e)}")
        return None

# Loop through each dataframe
for i in range(1, 41):
    # Dynamically generate dataframe name
    df_name = f'imdb_movies_df_{i}'
    # Assuming dataframes are stored in a dictionary or similar structure
    df = globals()[df_name]  # Replace with the actual way to access the dataframe

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        movie_name = row['Movie Name']
        release_date = row['Release Date']

        # Get User Reviews URL for the movie
        user_reviews_url = get_user_reviews_url(movie_name, release_date)

        # Update DataFrame with the URL
        df.at[index, 'User Reviews URL'] = user_reviews_url

    # Save the updated DataFrame
    output_path = f'/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/IMDb_movie_review_sentiment_analysis/20_movies_datasets_with_URL/imdb_movies_df_{i}_url.csv'
    df.to_csv(output_path, index=False)

# Close the browser
driver.quit()

-----

# Descriptive stats for all URL df

In [377]:
import os
import pandas as pd

# import all 20 csv files from the folder location '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/IMDb_movie_review_sentiment_analysis/20_movies_datasets_with_URL' The files are named imdb_movies_df_1_url.csv, imdb_movies_df_2_url.csv, imdb_movies_df_3_url.csv, and so on till 20. I want to concatenate all these files into one dataframe called all_URL_df

# Define the directory where the CSV files are located
directory = '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/IMDb_movie_review_sentiment_analysis/20_movies_datasets_with_URL'

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through all the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Append the DataFrame to the list
        dataframes.append(df)

# Concatenate all the DataFrames in the list
all_URL_df = pd.concat(dataframes, ignore_index=True)

In [378]:
# Display the final table
all_URL_df.head()

,Movie Name,Genre,Rating,Release Date,isAdult,Runtime Minutes,numVotes,User Reviews URL
0,Prey,"Action,Adventure,Drama",7.1,2022,0,100.0,222499,https://www.imdb.com/title/tt11866324/reviews/...
1,The Bye Bye Man,"Drama,Fantasy,Horror",4.3,2017,0,96.0,25774,https://www.imdb.com/title/tt4030600/reviews/?...
2,Fyre,"Crime,Documentary,Music",7.2,2019,0,97.0,51546,https://www.imdb.com/title/tt9412098/reviews/?...
3,Escape Room,"Action,Adventure,Horror",6.4,2019,0,99.0,139759,https://www.imdb.com/title/tt5886046/reviews/?...
4,Salaar,"Action,Crime,Drama",6.5,2023,0,175.0,57702,https://www.imdb.com/title/tt13927994/reviews/...


In [379]:
all_URL_df.shape

(1100, 8)

In [380]:
# all null values in each column
all_URL_df.isnull().sum()

Movie Name           0
Genre                0
Rating               0
Release Date         0
isAdult              0
Runtime Minutes      1
numVotes             0
User Reviews URL    69
dtype: int64

In [ ]:
# # drop all rows with null values in the User Reviews URL column
# all_URL_df = all_URL_df.dropna(subset=['User Reviews URL'])

In [381]:
# set datatypes for all_URL_df all columns
all_URL_df['Movie Name'] = all_URL_df['Movie Name'].astype('string')
all_URL_df['Genre'] = all_URL_df['Genre'].astype('string')
all_URL_df['Rating'] = all_URL_df['Rating'].astype('float')
all_URL_df['Release Date'] = all_URL_df['Release Date'].astype('int')
all_URL_df['isAdult'] = all_URL_df['isAdult'].astype('int')
all_URL_df['Runtime Minutes'] = all_URL_df['Runtime Minutes'].astype('float')
all_URL_df['numVotes'] = all_URL_df['numVotes'].astype('float')
all_URL_df['User Reviews URL'] = all_URL_df['User Reviews URL'].astype('string')

-----

## Loading all 20 URL csv as Dataframes

In [34]:
# load all 20 csv from the location '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/IMDb_movie_review_sentiment_analysis/20_movies_datasets_with_URL' as individual dataframes with the same name as their file name but without the .csv extension

import os
import pandas as pd

# Define the directory where the CSV files are located
directory = '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/IMDb_movie_review_sentiment_analysis/20_movies_datasets_with_URL'

# Initialize an empty dictionary to store the dataframes
dataframes = {}

# Loop through all the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Remove the '.csv' extension and use the filename as the dictionary key
        dataframe_name = filename.replace('.csv', '')
        # Store the DataFrame in the dictionary
        dataframes[dataframe_name] = df

In [ ]:
# Defining all dataframes as variables
for i in range(1, 24):
    globals()[f"imdb_movies_df_{i}_url"] = dataframes[f"imdb_movies_df_{i}_url"]

# Scrapping IMDb website code
-----

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import time
import re
import traceback
import os

# Initialize WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)

def scrape_reviews(url):
    if not url:  # Check if the URL is None or empty and skip if true
        return []
    driver.get(url)
    reviews_data = []
    scraped_reviews = set()  # To track scraped reviews and avoid duplicates
    
    try:
        while len(reviews_data) < 1000:  # Limit to 1000 reviews
            wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#main > section > div.lister > div.lister-list")))
            time.sleep(3)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            review_elements = soup.select("#main > section > div.lister > div.lister-list > div")
            if not review_elements:
                break
            for review_element in review_elements:
                if len(reviews_data) >= 1000:  # Check if we have already collected 1000 reviews
                    break
                try:
                    individual_rating_element = review_element.select_one(".ipl-ratings-bar > span > span:nth-child(2)")
                    individual_rating = int(individual_rating_element.text.strip()) if individual_rating_element else None
                    review_data = {
                        'Review Date': review_element.select_one(".review-date").text.strip(),
                        'Review Title': review_element.select_one(".title").text.strip(),
                        'Username': review_element.select_one(".display-name-link > a").text.strip(),
                        'Helpfulness': review_element.select_one(".actions.text-muted").text.strip(),
                        'Helpful Votes': int(re.findall(r'(\d+)', review_element.select_one(".actions.text-muted").text.strip())[0]),
                        'Total Votes': int(re.findall(r'(\d+)', review_element.select_one(".actions.text-muted").text.strip())[1]),
                        'Individual Rating': individual_rating,
                        'Review Text': review_element.select_one(".text.show-more__control").text.strip(),
                        'Spoiler Warning': review_element.select_one(".spoiler-warning").text.strip() if review_element.select_one(".spoiler-warning") else ""
                    }
                    review_identifier = (review_data['Review Date'], review_data['Username'])
                    if review_identifier not in scraped_reviews:
                        reviews_data.append(review_data)
                        scraped_reviews.add(review_identifier)
                except Exception as e:
                    print(f"Error occurred while extracting review data: {e}")
                    traceback.print_exc()
            try:
                load_more_button = wait.until(EC.element_to_be_clickable((By.ID, "load-more-trigger")))
                driver.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
                load_more_button.click()
                time.sleep(3)  # Adjust sleep time as necessary
            except TimeoutException:
                print("No more reviews to load.")
                break
    except Exception as e:
        print(f"Error occurred: {e}")
        traceback.print_exc()
    finally:
        return reviews_data

# Directory to save CSV files
save_dir = '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/IMDb_movie_review_sentiment_analysis/20_movies_datasets_with_Scrapped_data'

# Main loop to iterate through dataframes
for i in range(1, 41):  # Adjust range as necessary
    df_name = f'imdb_movies_df_{i}_url'
    df = eval(df_name)  # Dynamically get the dataframe
    all_reviews = []
    for index, row in df.iterrows():
        if pd.notnull(row['User Reviews URL']):  # Check if the URL is not null
            movie_reviews = scrape_reviews(row['User Reviews URL'])
            for review in movie_reviews:
                movie_data = row.to_dict()
                movie_data['Rating'] = movie_data.get('Rating')  # Ensure the 'Rating' key is from the dataframe
                review.update(movie_data)  # Add movie data to each review
            all_reviews.extend(movie_reviews)
    
    # Convert to DataFrame
    final_df = pd.DataFrame(all_reviews)
    
    # Save to CSV
    csv_file_path = os.path.join(save_dir, f'all_scrapped_data_{i}.csv')
    final_df.to_csv(csv_file_path, index=False)
    print(f"Dataframe {i} reviews saved to {csv_file_path}")

driver.quit()

# Importing all 20 scrapped CSV's and concatenating them
-----

In [295]:
# Import all 20 csv files from the folder location '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/IMDb_movie_review_sentiment_analysis/20_movies_datasets_with_Scrapped_data' The files are named all_scrapped_data_1.csv, all_scrapped_data_2.csv, all_scrapped_data_3.csv, and so on till 20. I want to concatenate all these files into one dataframe called all_reviews_df. First merge the first 2 then clear the memory and then merge the next one with the just merged one and so on till all 20 are merged.

import os
import pandas as pd

# Define the directory where the CSV files are located
directory = '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/IMDb_movie_review_sentiment_analysis/20_movies_datasets_with_Scrapped_data'

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through all the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Append the DataFrame to the list
        dataframes.append(df)

# Concatenate all the DataFrames in the list
all_reviews_df = pd.concat(dataframes, ignore_index=True)

In [296]:
all_reviews_df.head(2)

,Review Date,Review Title,Username,Helpfulness,Helpful Votes,Total Votes,Individual Rating,Review Text,Spoiler Warning,Movie Name,Genre,Rating,Release Date,isAdult,Runtime Minutes,numVotes,User Reviews URL
0,9 March 2023,Better than I expected.,deloudelouvain,44 out of 54 found this helpful.\n ...,44,54,8.0,Sharper didn't fail to entertain me. A perfect...,NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...
1,22 February 2023,Who's conning whom exactly?,paul-allaer,37 out of 46 found this helpful.\n ...,37,46,6.0,"As ""Sharper"" (2023 release; 116 min.) opens, w...",NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...
2,19 February 2023,I liked this more than I expected to,bk753,88 out of 111 found this helpful.\n ...,88,111,8.0,I have to stop coming here and using User Revi...,NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...
3,19 February 2023,In a genre that is hard to succeed in these da...,jtindahouse,51 out of 74 found this helpful.\n ...,51,74,8.0,'Sharper' is my kind of movie. But there's a r...,NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...
4,11 February 2023,A Stylish Thriller That Misses the Mark,FilmFanatic2023,84 out of 123 found this helpful.\n ...,84,123,6.0,A stylish thriller that tries to mimic the sty...,NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...


In [297]:
all_reviews_df.shape

(1016245, 17)

In [299]:
# Number of all unique values in the Movie name column in all_reviews_df
all_reviews_df['Movie Name'].nunique()

1027

In [300]:
# How many times a value appears in the Movie name column in all_reviews_df
repeats = all_reviews_df['Movie Name'].value_counts()

repeats.head(25)

Movie Name
Beast                                       2000
The Guilty                                  2000
Close                                       2000
The Good Neighbor                           2000
Kimi                                        1000
The Wife                                    1000
The Royal Treatment                         1000
Waiting for the Barbarians                  1000
Captain Underpants: The First Epic Movie    1000
Overboard                                   1000
Loving Vincent                              1000
Gretel & Hansel                             1000
Sand Castle                                 1000
Song to Song                                1000
Swiped                                      1000
The Night Eats the World                    1000
Tag                                         1000
Expend4bles                                 1000
Pet Sematary: Bloodlines                    1000
A Family Man                                1000
Uruttu Te

In [301]:
# filter repeats for values more than 999
repeats = repeats[repeats > 999]

In [302]:
repeats.shape

(1012,)

# Cleaning the all_reviews_df dataset
--------

In [303]:
# all_reviews_df.head(5)

,Review Date,Review Title,Username,Helpfulness,Helpful Votes,Total Votes,Individual Rating,Review Text,Spoiler Warning,Movie Name,Genre,Rating,Release Date,isAdult,Runtime Minutes,numVotes,User Reviews URL
0,9 March 2023,Better than I expected.,deloudelouvain,44 out of 54 found this helpful.\n ...,44,54,8.0,Sharper didn't fail to entertain me. A perfect...,NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...
1,22 February 2023,Who's conning whom exactly?,paul-allaer,37 out of 46 found this helpful.\n ...,37,46,6.0,"As ""Sharper"" (2023 release; 116 min.) opens, w...",NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...
2,19 February 2023,I liked this more than I expected to,bk753,88 out of 111 found this helpful.\n ...,88,111,8.0,I have to stop coming here and using User Revi...,NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...
3,19 February 2023,In a genre that is hard to succeed in these da...,jtindahouse,51 out of 74 found this helpful.\n ...,51,74,8.0,'Sharper' is my kind of movie. But there's a r...,NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...
4,11 February 2023,A Stylish Thriller That Misses the Mark,FilmFanatic2023,84 out of 123 found this helpful.\n ...,84,123,6.0,A stylish thriller that tries to mimic the sty...,NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...


In [304]:
# # In the all_reviews_df dataframe remove all rows for the following movies - Beast, The Guilty, Close and The Good Neighbor where if the values in the Helpful Votes, Total Votes, Review Text, Rating and Review Title are the same then remove those rows for each movie.
# all_reviews_df = all_reviews_df[~((all_reviews_df['Movie Name'] == 'Beast') & (all_reviews_df.duplicated(subset=['Helpful Votes', 'Total Votes', 'Review Text', 'Rating', 'Review Title'])))]
# all_reviews_df = all_reviews_df[~((all_reviews_df['Movie Name'] == 'The Guilty') & (all_reviews_df.duplicated(subset=['Helpful Votes', 'Total Votes', 'Review Text', 'Rating', 'Review Title'])))]
# all_reviews_df = all_reviews_df[~((all_reviews_df['Movie Name'] == 'Close') & (all_reviews_df.duplicated(subset=['Helpful Votes', 'Total Votes', 'Review Text', 'Rating', 'Review Title'])))]
# all_reviews_df = all_reviews_df[~((all_reviews_df['Movie Name'] == 'The Good Neighbor') & (all_reviews_df.duplicated(subset=['Helpful Votes', 'Total Votes', 'Review Text', 'Rating', 'Review Title'])))]

In [305]:
# # Number of times a value appears in the Movie name column in all_reviews_df
# repeats = all_reviews_df['Movie Name'].value_counts()

# repeats.head(25)

Movie Name
Sharper                                     1000
The Wife                                    1000
Isn't It Romantic                           1000
The Royal Treatment                         1000
Waiting for the Barbarians                  1000
Captain Underpants: The First Epic Movie    1000
Overboard                                   1000
Loving Vincent                              1000
Gretel & Hansel                             1000
Sand Castle                                 1000
Song to Song                                1000
Swiped                                      1000
The Night Eats the World                    1000
Tag                                         1000
Expend4bles                                 1000
Pet Sematary: Bloodlines                    1000
Shut In                                     1000
Uruttu Tech                                 1000
Malignant                                   1000
Barbarian                                   1000
The Unbea

In [306]:
# ## Count of movies that have less than 1000 reviews.
# # count all movies in a new dataframe that have less than 1000 reviews
# less_than_1000 = all_reviews_df['Movie Name'].value_counts()
# less_than_1000 = less_than_1000[less_than_1000 < 1000]
# less_than_1000.shape

(19,)

In [307]:
# # List of all the movies that have less than 1000 reviews.
# less_than_1000

Movie Name
Beast                       400
The Guilty                  399
Close                       315
The Good Neighbor           205
Bairavaa                     25
Playing with Fire            24
MobKing                      24
Like Crazy                   22
Two Lottery Tickets          21
Smolensk                     21
The Cost of Deception        21
Fauci                        19
Recep Ivedik 5               18
C'est la vie!                16
Goodbye Berlin               15
Kaiva                         8
15/07: Break of Dawn          5
Turnabout                     5
At the End of the Tunnel      1
Name: count, dtype: int64

In [308]:
# # Drop the Helpfulness column from all_reviews_df
# all_reviews_df.drop('Helpfulness', axis=1, inplace=True)

In [309]:
# # Drop all movies that have less than 1000 reviews from all_reviews_df
# all_reviews_df = all_reviews_df[~all_reviews_df['Movie Name'].isin(less_than_1000.index)]

------

## Adding additional columns from IMDb dataset

In [310]:
import pandas as pd

# Define file paths for the datasets
basics_path = '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/Data/IMDb title basics.tsv'
ratings_path = '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/Data/IMDb Title Ratings.tsv'
akas_path = '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/Data/IMDb Title AKAs.tsv'

# Load the datasets
basics_df = pd.read_csv(basics_path, sep='\t', usecols=['tconst', 'titleType', 'primaryTitle', 'genres', 'isAdult', 'startYear', 'runtimeMinutes'])
ratings_df = pd.read_csv(ratings_path, sep='\t', usecols=['tconst', 'averageRating', 'numVotes'])
akas_df = pd.read_csv(akas_path, sep='\t', usecols=['titleId', 'region', 'language', 'types'])

# Rename 'titleId' to 'tconst' in akas_df for consistency
akas_df.rename(columns={'titleId': 'tconst'}, inplace=True)

# Merge the DataFrames on 'tconst'
merged_df = basics_df.merge(ratings_df, on='tconst', how='left')
merged_df = merged_df.merge(akas_df, on='tconst', how='left')

# Filter the types column for movies only
merged_df = merged_df[merged_df['titleType'] == 'movie']

# Convert the 'startYear' column to numeric
merged_df['startYear'] = pd.to_numeric(merged_df['startYear'], errors='coerce')

# Convert the 'numVotes' column to strings
merged_df['numVotes'] = merged_df['numVotes'].astype(str)

# Group the data by 'primaryTitle', 'numVotes', and 'startYear' and concatenate the values in the other columns
merged_df = merged_df.groupby(['primaryTitle', 'numVotes', 'startYear']).agg({
    'region': lambda x: ','.join(x.astype(str)),
    'language': lambda x: ','.join(x.astype(str)),
    'types': lambda x: ','.join(x.astype(str)),
}).reset_index()

# Rename primaryTitle to Movie Name
merged_df.rename(columns={'primaryTitle': 'Movie Name'}, inplace=True)

# Rename startYear to Release Date
merged_df.rename(columns={'startYear': 'Release Date'}, inplace=True)

/var/folders/7_/z9jxc0jj6fx4snlcjg1wby4r0000gn/T/ipykernel_11102/335641486.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  basics_df = pd.read_csv(basics_path, sep='\t', usecols=['tconst', 'titleType', 'primaryTitle', 'genres', 'isAdult', 'startYear', 'runtimeMinutes'])


In [311]:
# data types for movie_name, numVotes, Release Date in merged_df
merged_df['Movie Name'] = merged_df['Movie Name'].astype(str)
merged_df['numVotes'] = merged_df['numVotes'].astype(str)
merged_df['Release Date'] = merged_df['Release Date'].astype(str)

# remove the decimal from numVotes and release date
merged_df['numVotes'] = merged_df['numVotes'].str.replace('.0', '')
merged_df['Release Date'] = merged_df['Release Date'].str.replace('.0', '')

In [312]:
merged_df.shape

(573002, 6)

In [313]:
# data types for movie_name, numVotes, Release Date in merged_df
merged_df['Movie Name'] = merged_df['Movie Name'].astype(str)
merged_df['numVotes'] = merged_df['numVotes'].astype(str)
merged_df['Release Date'] = merged_df['Release Date'].astype(str)

# set data types for movie_name, numVotes, Release Date in all_reviews_df
all_reviews_df['Movie Name'] = all_reviews_df['Movie Name'].astype(str)
all_reviews_df['numVotes'] = all_reviews_df['numVotes'].astype(str)
all_reviews_df['Release Date'] = all_reviews_df['Release Date'].astype(str)

In [314]:
# Merge the DataFrames merged_df and all_reviews_df if the Movie name, numVotes and Release Date are the same in both DataFrames then bring the region, language, types and isOriginalTitle from the merged_df to all_reviews_df
merged_df = all_reviews_df.merge(merged_df, on=['Movie Name', 'numVotes', 'Release Date'], how='left')

# Display the final table
merged_df.head(5)

,Review Date,Review Title,Username,Helpful Votes,Total Votes,Individual Rating,Review Text,Spoiler Warning,Movie Name,Genre,Rating,Release Date,isAdult,Runtime Minutes,numVotes,User Reviews URL,region,language,types
0,9 March 2023,Better than I expected.,deloudelouvain,44,54,8.0,Sharper didn't fail to entertain me. A perfect...,NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...,"RU,VN,ZA,ES,MX,AR,IT,FR,IN,RS,HU,AU,UA,TW,JP,G...","\N,\N,en,\N,\N,\N,\N,\N,en,\N,\N,\N,\N,\N,ja,\...","imdbDisplay,imdbDisplay,imdbDisplay,imdbDispla..."
1,22 February 2023,Who's conning whom exactly?,paul-allaer,37,46,6.0,"As ""Sharper"" (2023 release; 116 min.) opens, w...",NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...,"RU,VN,ZA,ES,MX,AR,IT,FR,IN,RS,HU,AU,UA,TW,JP,G...","\N,\N,en,\N,\N,\N,\N,\N,en,\N,\N,\N,\N,\N,ja,\...","imdbDisplay,imdbDisplay,imdbDisplay,imdbDispla..."
2,19 February 2023,I liked this more than I expected to,bk753,88,111,8.0,I have to stop coming here and using User Revi...,NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...,"RU,VN,ZA,ES,MX,AR,IT,FR,IN,RS,HU,AU,UA,TW,JP,G...","\N,\N,en,\N,\N,\N,\N,\N,en,\N,\N,\N,\N,\N,ja,\...","imdbDisplay,imdbDisplay,imdbDisplay,imdbDispla..."
3,19 February 2023,In a genre that is hard to succeed in these da...,jtindahouse,51,74,8.0,'Sharper' is my kind of movie. But there's a r...,NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...,"RU,VN,ZA,ES,MX,AR,IT,FR,IN,RS,HU,AU,UA,TW,JP,G...","\N,\N,en,\N,\N,\N,\N,\N,en,\N,\N,\N,\N,\N,ja,\...","imdbDisplay,imdbDisplay,imdbDisplay,imdbDispla..."
4,11 February 2023,A Stylish Thriller That Misses the Mark,FilmFanatic2023,84,123,6.0,A stylish thriller that tries to mimic the sty...,NaN,Sharper,"Crime,Drama,Thriller",6.7,2023,0,116.0,29167,https://www.imdb.com/title/tt12573454/reviews/...,"RU,VN,ZA,ES,MX,AR,IT,FR,IN,RS,HU,AU,UA,TW,JP,G...","\N,\N,en,\N,\N,\N,\N,\N,en,\N,\N,\N,\N,\N,ja,\...","imdbDisplay,imdbDisplay,imdbDisplay,imdbDispla..."


In [315]:
merged_df.shape

(1008000, 19)

-------

# Creating dummy variables

In [316]:
# In the merged_df dataframe create dummy variables for the region column in the merged_df dataframe. the regions are separated by commas. The dummy variables should be 1 if the region is present and 0 if not. The dummy variables should be named after the region with the prefix 'region_'.

# Split the 'region' column by commas and expand the split strings into separate columns
region_dummies = merged_df['region'].str.get_dummies(sep=',')
# Add a prefix to the column names
region_dummies = region_dummies.add_prefix('region_')

# Concatenate the dummy variables to the original DataFrame
merged_df = pd.concat([merged_df, region_dummies], axis=1)

In [317]:
# In the merged_df dataframe create dummy variables for the language column in the merged_df dataframe. the languages are separated by commas. The dummy variables should be 1 if the language is present and 0 if not. The dummy variables should be named after the language with the prefix 'language_'.

# Split the 'language' column by commas and expand the split strings into separate columns
language_dummies = merged_df['language'].str.get_dummies(sep=',')
# Add a prefix to the column names
language_dummies = language_dummies.add_prefix('language_')

# Concatenate the dummy variables to the original DataFrame
merged_df = pd.concat([merged_df, language_dummies], axis=1)

In [318]:
# In the merged_df dataframe create dummy variables for the language column in the merged_df dataframe. the languages are separated by commas. The dummy variables should be 1 if the language is present and 0 if not. The dummy variables should be named after the language with the prefix 'language_'.

# Split the 'language' column by commas and expand the split strings into separate columns
types_dummies = merged_df['types'].str.get_dummies(sep=',')
# Add a prefix to the column names
types_dummies = types_dummies.add_prefix('types_')

# Concatenate the dummy variables to the original DataFrame
merged_df = pd.concat([merged_df, types_dummies], axis=1)

In [319]:
# In the merged_df dataframe create dummy variables for the language column in the merged_df dataframe. the languages are separated by commas. The dummy variables should be 1 if the language is present and 0 if not. The dummy variables should be named after the language with the prefix 'language_'.

# Split the 'language' column by commas and expand the split strings into separate columns
Genre_dummies = merged_df['Genre'].str.get_dummies(sep=',')
# Add a prefix to the column names
Genre_dummies = Genre_dummies.add_prefix('Genre_')

# Concatenate the dummy variables to the original DataFrame
merged_df = pd.concat([merged_df, Genre_dummies], axis=1)

In [320]:
# Create a dummy variable for the 'Spoiler Warning' column in the merged_df dataframe. The dummy variable should be 1 if the 'Spoiler Warning' column has the value 'Warning: Spoilers' else it will be 0.

# Create a dummy variable for the 'Spoiler Warning' column
merged_df['Spoiler Warning'] = (merged_df['Spoiler Warning'] == 'Warning: Spoilers').astype(int)

In [321]:
merged_df.head(5)

,Review Date,Review Title,Username,Helpful Votes,Total Votes,Individual Rating,Review Text,Spoiler Warning,Movie Name,Genre,...,Genre_Music,Genre_Musical,Genre_Mystery,Genre_News,Genre_Romance,Genre_Sci-Fi,Genre_Sport,Genre_Thriller,Genre_War,Genre_Western
0,9 March 2023,Better than I expected.,deloudelouvain,44,54,8.0,Sharper didn't fail to entertain me. A perfect...,0,Sharper,"Crime,Drama,Thriller",...,0,0,0,0,0,0,0,1,0,0
1,22 February 2023,Who's conning whom exactly?,paul-allaer,37,46,6.0,"As ""Sharper"" (2023 release; 116 min.) opens, w...",0,Sharper,"Crime,Drama,Thriller",...,0,0,0,0,0,0,0,1,0,0
2,19 February 2023,I liked this more than I expected to,bk753,88,111,8.0,I have to stop coming here and using User Revi...,0,Sharper,"Crime,Drama,Thriller",...,0,0,0,0,0,0,0,1,0,0
3,19 February 2023,In a genre that is hard to succeed in these da...,jtindahouse,51,74,8.0,'Sharper' is my kind of movie. But there's a r...,0,Sharper,"Crime,Drama,Thriller",...,0,0,0,0,0,0,0,1,0,0
4,11 February 2023,A Stylish Thriller That Misses the Mark,FilmFanatic2023,84,123,6.0,A stylish thriller that tries to mimic the sty...,0,Sharper,"Crime,Drama,Thriller",...,0,0,0,0,0,0,0,1,0,0


----

### Additional null value cleaning in the dataset

In [322]:
# Drop the following columns User Reviews URL, region, language, types
merged_df.drop(['User Reviews URL', 'region', 'language', 'types'], axis=1, inplace=True)

In [323]:
# Drop all rows with the missing value in Review Title and Individual Rating columns in the merged_df dataframe
merged_df = merged_df[merged_df['Review Title'].notnull()]
merged_df = merged_df[merged_df['Individual Rating'].notnull()]

-----

### Removing all rows with the same Review Date, Review Title, Username, Review Text, Spoiler Warning, Movie Name, numVotes, Release Date, Rating, Runtime Minutes, isAdult, and Genre

In [324]:
# in merged_df if there are rows with the same Review Date, Review Title, Username, Review Text, Spoiler Warning, Movie Name, numVotes, Release Date, Rating, Runtime Minutes, isAdult, and Genre then remove all but keep one out of them.

# Remove all rows with the same Review Date, Review Title, Username, Review Text, Spoiler Warning, Movie Name, numVotes, Release Date, Rating, Runtime Minutes, isAdult, and Genre
merged_df = merged_df.drop_duplicates(subset=['Review Date', 'Review Title', 'Username', 'Review Text', 'Movie Name', 'numVotes', 'Release Date', 'Rating', 'Runtime Minutes'])

# Reset index
merged_df.reset_index(drop=True, inplace=True)

----

In [325]:
# export the all_reviews_df dataframe to a csv file in the location '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/all_reviews_df.csv'
merged_df.to_csv('/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/all_reviews_df.csv', index=False)

In [326]:
# Clear all variables and memory
%reset -f

# Final dataframe is called **Movie_reviews_df**
---------

In [394]:
import pandas as pd

# load all_reviews_df.csv into a dataframe called all_reviews_df
all_reviews_df = pd.read_csv('/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/all_reviews_df.csv')

In [395]:
all_reviews_df.head(2)

,Review Date,Review Title,Username,Helpful Votes,Total Votes,Individual Rating,Review Text,Spoiler Warning,Movie Name,Genre,...,Genre_Music,Genre_Musical,Genre_Mystery,Genre_News,Genre_Romance,Genre_Sci-Fi,Genre_Sport,Genre_Thriller,Genre_War,Genre_Western
0,9 March 2023,Better than I expected.,deloudelouvain,44,54,8.0,Sharper didn't fail to entertain me. A perfect...,0,Sharper,"Crime,Drama,Thriller",...,0,0,0,0,0,0,0,1,0,0
1,22 February 2023,Who's conning whom exactly?,paul-allaer,37,46,6.0,"As ""Sharper"" (2023 release; 116 min.) opens, w...",0,Sharper,"Crime,Drama,Thriller",...,0,0,0,0,0,0,0,1,0,0
2,19 February 2023,I liked this more than I expected to,bk753,88,111,8.0,I have to stop coming here and using User Revi...,0,Sharper,"Crime,Drama,Thriller",...,0,0,0,0,0,0,0,1,0,0
3,19 February 2023,In a genre that is hard to succeed in these da...,jtindahouse,51,74,8.0,'Sharper' is my kind of movie. But there's a r...,0,Sharper,"Crime,Drama,Thriller",...,0,0,0,0,0,0,0,1,0,0
4,11 February 2023,A Stylish Thriller That Misses the Mark,FilmFanatic2023,84,123,6.0,A stylish thriller that tries to mimic the sty...,0,Sharper,"Crime,Drama,Thriller",...,0,0,0,0,0,0,0,1,0,0


In [396]:
all_reviews_df.columns

Index(['Review Date', 'Review Title', 'Username', 'Helpful Votes',
       'Total Votes', 'Individual Rating', 'Review Text', 'Spoiler Warning',
       'Movie Name', 'Genre',
       ...
       'Genre_Music', 'Genre_Musical', 'Genre_Mystery', 'Genre_News',
       'Genre_Romance', 'Genre_Sci-Fi', 'Genre_Sport', 'Genre_Thriller',
       'Genre_War', 'Genre_Western'],
      dtype='object', length=213)

In [397]:
# Save all the all_reviews_df columns as a list
all_reviews_df.columns.to_list()

['Review Date',
 'Review Title',
 'Username',
 'Helpful Votes',
 'Total Votes',
 'Individual Rating',
 'Review Text',
 'Spoiler Warning',
 'Movie Name',
 'Genre',
 'Rating',
 'Release Date',
 'isAdult',
 'Runtime Minutes',
 'numVotes',
 'region_AE',
 'region_AL',
 'region_AM',
 'region_AR',
 'region_AT',
 'region_AU',
 'region_AZ',
 'region_BA',
 'region_BD',
 'region_BE',
 'region_BG',
 'region_BJ',
 'region_BO',
 'region_BR',
 'region_BS',
 'region_BY',
 'region_CA',
 'region_CG',
 'region_CH',
 'region_CL',
 'region_CM',
 'region_CN',
 'region_CO',
 'region_CR',
 'region_CSHH',
 'region_CU',
 'region_CZ',
 'region_DE',
 'region_DK',
 'region_DO',
 'region_DZ',
 'region_EC',
 'region_EE',
 'region_EG',
 'region_ES',
 'region_FI',
 'region_FR',
 'region_GB',
 'region_GE',
 'region_GL',
 'region_GR',
 'region_GT',
 'region_HK',
 'region_HR',
 'region_HU',
 'region_ID',
 'region_IE',
 'region_IL',
 'region_IN',
 'region_IR',
 'region_IS',
 'region_IT',
 'region_JM',
 'region_JP',
 'regi

In [398]:
all_reviews_df.dtypes

Review Date       object
Review Title      object
Username          object
Helpful Votes      int64
Total Votes        int64
                   ...  
Genre_Sci-Fi       int64
Genre_Sport        int64
Genre_Thriller     int64
Genre_War          int64
Genre_Western      int64
Length: 213, dtype: object

In [399]:
all_reviews_df.shape

(165553, 213)

# Analysis below
-----

In [400]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define the feature and target variables
X = all_reviews_df['Review Text']
y = all_reviews_df['Individual Rating']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the training data and transform both training and testing data
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Initialize the Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train_vect, y_train)

# Predict the ratings on the test set
y_pred = rf.predict(X_test_vect)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Define the threshold for big discrepancies
threshold = 2

# Find reviews with a big discrepancy between predicted and actual ratings
discrepancies = pd.DataFrame({'Review': X_test, 'Actual Rating': y_test, 'Predicted Rating': y_pred})
discrepancies['Discrepancy'] = discrepancies['Actual Rating'] - discrepancies['Predicted Rating']
big_discrepancies = discrepancies[discrepancies['Discrepancy'].abs() > threshold]

# Display the reviews with big discrepancies
print(big_discrepancies)

In [ ]:

# save big_discrepancies to a csv file in the location '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - EECONOMIC ANALYTICS II/Project Code file/big_discrepancies.csv'
big_discrepancies.to_csv('/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/big_discrepancies.csv', index=False)

In [401]:
# load the big_discrepancies.csv into a dataframe called big_discrepancies
big_discrepancies = pd.read_csv('/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/2nd Semester - MSEA/ECON 5823 - ECONOMIC ANALYTICS II/Project Code file/big_discrepancies.csv')

In [402]:
big_discrepancies.head(20)